In [1]:
from requests import get

In [2]:
BIG_FILE_URL = 'https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt'

In [3]:
with open('big.txt','wb') as big_f:
    response = get(BIG_FILE_URL, )
    
    if response.status_code == 200:
        big_f.write(response.content)
    else:
        print('error status : {}'.format(response.reason))

In [4]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE 
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

In [5]:
tokenizer = Tokenizer(BPE())

tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

tokenizer.pre_tokenizer = ByteLevel()

tokenizer.decoder = ByteLevelDecoder()

In [6]:
from tokenizers.trainers import BpeTrainer

In [7]:
trainer = BpeTrainer(vocab_size=25000,
                     show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(trainer,['big.txt'])
print('Trained vocab size:{}'.format(tokenizer.get_vocab_size()))

Trained vocab size:25000


In [8]:
tokenizer.model.save('.')

['.\\vocab.json', '.\\merges.txt']

In [87]:
tokenizer.model = BPE.from_file('vocab.json', 'merges.txt')
encoding = tokenizer.encode('this is the shortest one, so')

print('Encoding strings:{}'.format(encoding.tokens))
print(encoding.ids)
decoded = tokenizer.decode(encoding.ids)
print('Decode returns {}'.format(decoded))

Encoding strings:['Ġthis', 'Ġis', 'Ġthe', 'Ġshort', 'est', 'Ġone', ',', 'Ġso']
[431, 338, 260, 1825, 411, 459, 11, 389]
Decode returns  this is the shortest one, so


In [38]:
from transformers import AutoTokenizer

In [6]:
tknz = AutoTokenizer.from_pretrained('bert-large-cased')

In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer, top_k_top_p_filtering
from torch.nn import functional as F
import torch

In [19]:
input_ids = tknz(["Hello", "I'm", "a", "single", "sentence"], is_split_into_words=True)
print(type(input_ids))
print(input_ids)
for k in input_ids['input_ids']:
    print(tknz.decode(k),end='\n')

<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': [101, 8667, 146, 112, 182, 170, 1423, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
[ C L S ]
H e l l o
I
'
m
a
s i n g l e
s e n t e n c e
[ S E P ]


In [8]:
input_ids = tknz.encode('this is the shortest one i have seen', return_tensors="pt")
print(input_ids.tolist())
for k in input_ids.tolist()[0]:
    print(tknz.decode(k),end='\n')

[[101, 1142, 1110, 1103, 22710, 1141, 178, 1138, 1562, 102]]
[ C L S ]
t h i s
i s
t h e
s h o r t e s t
o n e
i
h a v e
s e e n
[ S E P ]


In [9]:
input_ids

tensor([[  101,  1142,  1110,  1103, 22710,  1141,   178,  1138,  1562,   102]])

In [95]:

# get logits of last hidden state
next_token_logits = model(input_ids).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = F.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)
print(generated)
resulting_string = tk.decode(generated.tolist()[0])

tensor([[ 5661,   318,   262, 35581,   530,    11,   523,   611]])


In [96]:
print(next_token)
print(tk.decode(next_token[0]))
print(resulting_string)

tensor([[611]])
 if
this is the shortest one, so if


In [21]:
sp_token = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
from transformers import AutoModelForSequenceClassification

In [103]:
bt = AutoTokenizer.from_pretrained('bert-base-uncased')

In [104]:
input_ids = bt.encode('this is the shortest one, so', return_tensors="pt")
print(input_ids.tolist())
for k in input_ids.tolist()[0]:
    print(bt.decode(k),end='\n')

[[101, 2023, 2003, 1996, 20047, 2028, 1010, 2061, 102]]
[ C L S ]
t h i s
i s
t h e
s h o r t e s t
o n e
,
s o
[ S E P ]


In [105]:
btm = AutoModelWithLMHead.from_pretrained('bert-base-uncased')

F:\Anaconda\envs\deepl\lib\site-packages\transformers\modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [115]:
next_token_logits = btm.generate(input_ids, max_length=10, do_sample=True, top_p=0.95, top_k=60)
print(next_token_logits)
tk.decode(next_token_logits[0])

tensor([[  101,  2023,  2003,  1996, 20047,  2028,  1010,  2061,   102,  1012]])


'�ober futureput emphasizedresenttersWhat� Cl'